In [1]:
# import library
import pickle
import pandas as pd
import numpy as np

import xgboost as xgb
import lightgbm as lgb
import time

from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from keras import models
from keras import layers
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Dense, GaussianNoise, GaussianDropout
from keras.models import Sequential, Model
from keras.regularizers import l2, l1
from keras.optimizers import Adam, SGD, Nadam
from keras.callbacks import LearningRateScheduler
from keras.metrics import categorical_accuracy
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from  keras.regularizers import l1, l2
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

import matplotlib.pyplot as plt

%matplotlib inline

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(
    gpu_options=tf.GPUOptions(
        visible_device_list="1", 
        allow_growth=True,
#         per_process_gpu_memory_fraction=0.5
    )
)
set_session(tf.Session(config=config))

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# load data
df = pd.concat([
    pd.read_csv('../data/feature_selection_positive.csv', index_col=0),
    pd.read_csv('../data/decomp_pos.csv', index_col=0).drop('Subclass', axis=1)
], axis=1)

In [3]:
# divide objective and target
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)

features = df.drop('Subclass', axis=1)

In [4]:
# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

In [5]:
# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [6]:
# define and fit
clf = rf()
clf = clf.fit(X_train, y_train)

# append accuracy to list
t = []
t.append(clf.score(X_test, y_test))
pickle.dump(clf, open('../model/rf_pos_fs+decomp.sav', 'wb'))

In [7]:
# setting params
params = {}
params['device'] = 'gpu'
params['gpu_id'] = 1
params['updater'] = 'grow_gpu_hist'

In [8]:
# define and fit
model = xgb.XGBClassifier(**params)
model.fit(X_train, y_train)

pickle.dump(model, open('../model/xgb_pos_fs+decomp.sav', 'wb'))
t.append(model.score(X_test, y_test))

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [9]:
# define and fit 
gbm = lgb.LGBMClassifier(
    objective='multiclass',
    device = 'gpu'
)

gbm.fit(
    X_train, 
    y_train,
    eval_set=[(X_test, y_test)],
    early_stopping_rounds=5
)
pickle.dump(gbm, open('../model/lgbm_pos_fs+decomp.sav', 'wb'))
t.append(gbm.score(X_test, y_test))

[1]	valid_0's multi_logloss: 1.2921
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's multi_logloss: 1.21684
[3]	valid_0's multi_logloss: 1.14773
[4]	valid_0's multi_logloss: 1.09008
[5]	valid_0's multi_logloss: 1.03823
[6]	valid_0's multi_logloss: 0.989192
[7]	valid_0's multi_logloss: 0.948878
[8]	valid_0's multi_logloss: 0.909039
[9]	valid_0's multi_logloss: 0.875592
[10]	valid_0's multi_logloss: 0.845211
[11]	valid_0's multi_logloss: 0.815212
[12]	valid_0's multi_logloss: 0.789262
[13]	valid_0's multi_logloss: 0.76775
[14]	valid_0's multi_logloss: 0.746653
[15]	valid_0's multi_logloss: 0.728312
[16]	valid_0's multi_logloss: 0.70798
[17]	valid_0's multi_logloss: 0.691939
[18]	valid_0's multi_logloss: 0.677652
[19]	valid_0's multi_logloss: 0.662271
[20]	valid_0's multi_logloss: 0.648085
[21]	valid_0's multi_logloss: 0.635502
[22]	valid_0's multi_logloss: 0.621209
[23]	valid_0's multi_logloss: 0.612667
[24]	valid_0's multi_logloss: 0.603786
[25]	valid_0's multi

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [10]:
tf.set_random_seed(42)

# make keras model
start = time.time()
inputs = Input(shape=(X_train.shape[1],))

x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# fit
epochs = 100
batch_size = 1000
es = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    X_train,
    y_train_for_keras,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test_for_keras),
    verbose=1,
    callbacks=[
        es,
#         lr_decay
    ]
)

model.save('../model/Keras_pos_fs+decomp.h5')
t.append(model.evaluate(X_test, y_test_for_keras, verbose=0)[1])

Train on 1340 samples, validate on 336 samples
Epoch 1/100
1340/1340 [==============================] - 6s 5ms/step - loss: 1.9534 - acc: 0.2590 - val_loss: 1.4771 - val_acc: 0.1964
Epoch 2/100
1340/1340 [==============================] - 0s 193us/step - loss: 1.7408 - acc: 0.2724 - val_loss: 1.4057 - val_acc: 0.2262
Epoch 3/100
1340/1340 [==============================] - 0s 181us/step - loss: 1.7276 - acc: 0.2799 - val_loss: 1.4272 - val_acc: 0.2500
Epoch 4/100
1340/1340 [==============================] - 0s 197us/step - loss: 1.6241 - acc: 0.2993 - val_loss: 1.4376 - val_acc: 0.2530
Epoch 5/100
1340/1340 [==============================] - 0s 225us/step - loss: 1.6144 - acc: 0.3090 - val_loss: 1.4151 - val_acc: 0.2887
Epoch 6/100
1340/1340 [==============================] - 0s 133us/step - loss: 1.5657 - acc: 0.3075 - val_loss: 1.3979 - val_acc: 0.3155
Epoch 7/100
1340/1340 [==============================] - 0s 199us/step - loss: 1.5497 - acc: 0.3187 - val_loss: 1.3740 - val_acc: 0.3

1340/1340 [==============================] - 0s 102us/step - loss: 0.5867 - acc: 0.7619 - val_loss: 0.8377 - val_acc: 0.6607
Epoch 61/100
1340/1340 [==============================] - 0s 155us/step - loss: 0.6042 - acc: 0.7493 - val_loss: 0.8696 - val_acc: 0.6369
Epoch 62/100
1340/1340 [==============================] - 0s 161us/step - loss: 0.5790 - acc: 0.7552 - val_loss: 0.8579 - val_acc: 0.6399
Epoch 63/100
1340/1340 [==============================] - 0s 161us/step - loss: 0.5950 - acc: 0.7545 - val_loss: 0.8158 - val_acc: 0.6607
Epoch 64/100
1340/1340 [==============================] - 0s 175us/step - loss: 0.5853 - acc: 0.7657 - val_loss: 0.8030 - val_acc: 0.6726
Epoch 65/100
1340/1340 [==============================] - 0s 253us/step - loss: 0.5448 - acc: 0.7896 - val_loss: 0.8280 - val_acc: 0.6607
Epoch 66/100
1340/1340 [==============================] - 0s 179us/step - loss: 0.5538 - acc: 0.7858 - val_loss: 0.8685 - val_acc: 0.6399
Epoch 67/100
1340/1340 [=======================

In [11]:
f = []
f.append(t)

In [12]:
f

[[0.7559523809523809,
  0.8214285714285714,
  0.8214285714285714,
  0.6964285714285714]]

In [13]:
# load data
df = pd.concat([
    pd.read_csv('../data/feature_selection_negative.csv', index_col=0),
    pd.read_csv('../data/decomp_neg.csv', index_col=0).drop('Subclass', axis=1)
], axis=1)

In [14]:
# divide objective and target
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)

features = df.drop('Subclass', axis=1)

In [15]:
# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

In [16]:
# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [17]:
# define and fit
clf = rf()
clf = clf.fit(X_train, y_train)

# append accuracy to list
t = []
t.append(clf.score(X_test, y_test))
pickle.dump(clf, open('../model/rf_ng_fs+decomp.sav', 'wb'))

In [18]:
# setting params
params = {}
params['device'] = 'gpu'
params['gpu_id'] = 1
params['updater'] = 'grow_gpu_hist'

In [19]:
# define and fit
model = xgb.XGBClassifier(**params)
model.fit(X_train, y_train)

pickle.dump(model, open('../model/xgb_ng_fs+decomp.sav', 'wb'))
t.append(model.score(X_test, y_test))

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [20]:
# define and fit 
gbm = lgb.LGBMClassifier(
    objective='multiclass',
    device = 'gpu'
)

gbm.fit(
    X_train, 
    y_train,
    eval_set=[(X_test, y_test)],
    early_stopping_rounds=5
)

pickle.dump(gbm, open('../model/lgbm_ng_fs+decomp.sav', 'wb'))
t.append(gbm.score(X_test, y_test))

[1]	valid_0's multi_logloss: 1.29626
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's multi_logloss: 1.21581
[3]	valid_0's multi_logloss: 1.14338
[4]	valid_0's multi_logloss: 1.08106
[5]	valid_0's multi_logloss: 1.03083
[6]	valid_0's multi_logloss: 0.983504
[7]	valid_0's multi_logloss: 0.945926
[8]	valid_0's multi_logloss: 0.916087
[9]	valid_0's multi_logloss: 0.885672
[10]	valid_0's multi_logloss: 0.854913
[11]	valid_0's multi_logloss: 0.833447
[12]	valid_0's multi_logloss: 0.810473
[13]	valid_0's multi_logloss: 0.785322
[14]	valid_0's multi_logloss: 0.768879
[15]	valid_0's multi_logloss: 0.754653
[16]	valid_0's multi_logloss: 0.736803
[17]	valid_0's multi_logloss: 0.721354
[18]	valid_0's multi_logloss: 0.708589
[19]	valid_0's multi_logloss: 0.701734
[20]	valid_0's multi_logloss: 0.687266
[21]	valid_0's multi_logloss: 0.674649
[22]	valid_0's multi_logloss: 0.663164
[23]	valid_0's multi_logloss: 0.655876
[24]	valid_0's multi_logloss: 0.647222
[25]	valid_0's mu

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [21]:
tf.set_random_seed(42)

# make keras model
start = time.time()
inputs = Input(shape=(X_train.shape[1],))

x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# fit
epochs = 100
batch_size = 1000
es = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    X_train,
    y_train_for_keras,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test_for_keras),
    verbose=1,
    callbacks=[
        es,
#         lr_decay
    ]
)

model.save('../model/Keras_ng_fs+decomp.h5')
t.append(model.evaluate(X_test, y_test_for_keras, verbose=0)[1])

Train on 373 samples, validate on 94 samples
Epoch 1/100
373/373 [==============================] - 6s 16ms/step - loss: 2.2294 - acc: 0.2252 - val_loss: 1.4013 - val_acc: 0.4043
Epoch 2/100
373/373 [==============================] - 0s 120us/step - loss: 1.8421 - acc: 0.3405 - val_loss: 1.3925 - val_acc: 0.4255
Epoch 3/100
373/373 [==============================] - 0s 176us/step - loss: 1.8700 - acc: 0.2708 - val_loss: 1.4616 - val_acc: 0.4468
Epoch 4/100
373/373 [==============================] - 0s 174us/step - loss: 1.6848 - acc: 0.3512 - val_loss: 1.5222 - val_acc: 0.4574
Epoch 5/100
373/373 [==============================] - 0s 160us/step - loss: 1.7106 - acc: 0.3003 - val_loss: 1.5856 - val_acc: 0.4574
Epoch 6/100
373/373 [==============================] - 0s 162us/step - loss: 1.6867 - acc: 0.3592 - val_loss: 1.5904 - val_acc: 0.4574
Epoch 7/100
373/373 [==============================] - 0s 164us/step - loss: 1.7134 - acc: 0.3324 - val_loss: 1.5525 - val_acc: 0.4574
Epoch 8/100

In [22]:
f.append(t)

In [24]:
pd.DataFrame(
    f, 
    columns=['Random Forest', 'XGBoost', 'LightGBM', 'Keras'], 
    index=['Feature Selection + Decomposition', 'Feature Selection + Decomposition']
)
# .to_csv('../result/Feature Selection + Decomposition.csv')

,Random Forest,XGBoost,LightGBM,Keras
Feature Selection + Decomposition,0.755952,0.821429,0.821429,0.696429
Feature Selection + Decomposition,0.702128,0.765957,0.808511,0.563830
